# Getting started with GitHub Actions

## About GitHub Actions

[GitHub Actions](https://docs.github.com/en/free-pro-team@latest/actions) help you automate tasks within your software development life cycle. GitHub say that:

> "*GitHub Actions are event-driven, meaning that you can run a series of commands after a specified event has occurred. For example, every time someone creates a pull request for a repository, you can automatically run a command that executes a software testing script.*"

They are a powerful system for automating many software development processes, and open source projects get unlimited compute hours for free! A sequence of steps in GitHub Actions are be combined together into a "workflow", which can furthermore contain multiple "jobs", which are run on multiple machines, with (if needed) multiple operating systems, in parallel. For more information, read GitHub's [Introduction to GitHub Actions](https://docs.github.com/en/free-pro-team@latest/actions/learn-github-actions/introduction-to-github-actions).

However, workflows can be difficult to create and debug, because:

- They run in a special environment which is difficult to replicate locally
- Each time you make a change it takes quite a few minutes before your updated workflow is run and you can see the results
- It is difficult to develop interatively and iteratively
- The actions environment and contexts are complex and the documentation, whilst detailed, provides few real examples to work from
- The main "language" for building workflows is a YAML-based language which is quite verbose, and doesn't allow you to leverage your knowledge of other programming languages
- Building full-featured extensions requires using TypeScript, which is not a language that most developers are used to using for scripting, sysadmin, and continuous integration tasks.

However, with `ghapi` you can write your workflows in Python and shell, and can do nearly all your development on your local machine. We'll start by importing from the library, along with `fastcore`:

In [ ]:
from ghapi import *
from fastcore.utils import *

## Your first Python-based workflow

We're going to help you get started by building a simple workflow which will add a comment to all new pull requests, saying "thank you" to the contributor.

`GhApi` makes developing workflows easier, because your actual YAML file is created for you entirely automatically. To create the new workflow, run the `create_workflow` function, like so:

In [ ]:
create_workflow(name='thankyou', event=Event.pull_request)

Now it's time to create our python script. You'll find it in `.github/scripts`, named based on your `create_workflow` parameters:

In [ ]:
path_scr = Path('.github/scripts/')
files = path_scr.ls(file_exts=['.py']); files

(#1) [Path('.github/scripts/build-thankyou-pull_request.py')]

The initial skeleton just contains import statements.

It will be much easier for us to iterate on our script if we have a sample *context* that the GitHub workflow runner will provide to us. We can get one by calling `example_payload`:

In [ ]:
example = example_payload(Event.pull_request)
list(example)

['action', 'number', 'pull_request', 'repository', 'sender']

For information about all the fields in a *payload*, use the GitHub [webhook payload](https://docs.github.com/en/free-pro-team@latest/developers/webhooks-and-events/webhook-events-and-payloads#pull_request) documentation. For instance, the docs tell us that the "action" field can be:

> "*opened, edited, closed, assigned, unassigned, review_requested, review_request_removed, ready_for_review, labeled, unlabeled, synchronize, locked, unlocked, or reopened*"

Let's see what it contains for our example payload:

In [ ]:
example.action

'opened'

For our script, we should ensure that it's only run for the `opened` action. Next up, we need to find out how to add a comment to a pull request. First, we'll need to create our `GhApi` object. On your own PC, your GitHub token should be in the `GITHUB_TOKEN` environment variable, whereas when run in a workflow it will be part of the `github` context. The `github_token` function handles this for you automatically, so we can say:

In [ ]:
api = GhApi(owner='fastai', repo='ghapi-test', token=github_token())

One way to find the correct operation to call is to search the [full API reference](https://ghapi.fast.ai/fullapi.html). Operations are generally named as `{verb}_{object}`, so search for `create_comment`. Alternatively, you can jump to the section that you expect to contain your required operation -- in this case, it's important to know that GitHub considers a "pull request" to be a special kind of "issue". After some looking through the page, we found this operation:

In [ ]:
api.issues.create_comment

[issues.create_comment](https://docs.github.com/rest/reference/issues#create-an-issue-comment)(issue_number, body): *Create an issue comment*

The hyperlink provided will take you to the GitHub docs for this operation, so take a look at that now. We need to provide an `issue_number` and the `body` of the comment. We can look inside the payload to find the issue number we need to use:

In [ ]:
', '.join(example.pull_request)

'url, id, node_id, html_url, diff_url, patch_url, issue_url, number, state, locked, title, user, body, created_at, updated_at, closed_at, merged_at, merge_commit_sha, assignee, assignees, requested_reviewers, requested_teams, labels, milestone, commits_url, review_comments_url, review_comment_url, comments_url, statuses_url, head, base, _links, author_association, draft, merged, mergeable, rebaseable, mergeable_state, merged_by, comments, review_comments, maintainer_can_modify, commits, additions, deletions, changed_files'

Since the docs call the parameter `issue_number`, and there is a `number` field here, that's probably what we should use.

To test this out, we'll first create a PR or issue, and then create our comment:

In [ ]:
com = api.issues.create_comment(issue_number=1, body='Thank you for your *valuable* contribution')

We can view the comment by visiting its URL:

In [ ]:
com.url

'https://api.github.com/repos/fastai/ghapi-test/issues/comments/737383236'

...and then delete it since we were just testing:

In [ ]:
com = api.issues.delete_comment(com.id)

The payload will be available in the [`github` context](https://docs.github.com/en/free-pro-team@latest/actions/reference/context-and-expression-syntax-for-github-actions#github-context), which is provided automatically to you using the `context_github` function. When running locally, an example github context will be used instead. (This example github context will not, in general, have the same payload as the event you're using, so use the `example payload` for that.)

The `event` contains the actual payload:

In [ ]:
list(context_github.event)

['inputs', 'organization', 'ref', 'repository', 'sender', 'workflow']

When we called `create_workflow`, the workflow it created can also be triggered using [workflow dispatch](https://github.blog/changelog/2020-07-06-github-actions-manual-triggers-with-workflow_dispatch/), which is more convenient for testing. In this case, our payload will not have the same information, so we should write our function in a way that can handle workflow dispatch as well.

In [ ]:
dispatch = example_payload(Event.workflow_dispatch)

We can check what type of trigger we're responding to, by checking the keys of our payload:

In [ ]:
'workflow' in dispatch

True

In that case, we'll use a fixed issue number, instead of using the actual payload number.

We can now write our function.

In [ ]:
def reply_thanks():
    payload = context_github.event
    if 'workflow' in payload: issue = 1
    else:
        if payload.action != 'opened': return
        api = GhApi(owner='fastai', repo='ghapi', token=github_token())
        issue = payload.number
    api.issues.create_comment(issue_number=issue, body='Thank you for your *valuable* contribution')

Finally, copy this to the script in `.github/scripts`, along with a line to run it (`reply_thanks()`), and commit it to GitHub.

, and try creating a pull request. Click on the "Actions" tab to watch your action run. If it doesn't work, try running the two lines at the end of the above function directly in a notebook or Python REPL, replacing `payload.number` with an actual issue or PR you create. You can also add `print(payload)` after the first line, and when GitHub runs your workflow you'll see the whole payload in your "Actions" panel.

#### TODO

- only if new contrib
- binder link if NB